# Linear Regression on Canada Per Capita Income (from scratch)

Implementation uses only NumPy for math and Matplotlib for visualization, following the standard gradient descent solution for univariate linear regression.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Ensure reproducibility
np.random.seed(42)

# Plot style
plt.style.use('seaborn-v0_8-whitegrid')

## 1. Load the CSV dataset
Dataset: `canada_per_capita_income.csv` (year, income).

In [2]:
def load_dataset(csv_path):
    """
    Load a two-column CSV (year, income) without using pandas.
    Returns feature column x (years) and target y (income), both as column vectors.
    """
    data = np.loadtxt(csv_path, delimiter=",", skiprows=1)
    x = data[:, 0:1]  # years
    y = data[:, 1:1+1]  # income
    return x, y

csv_path = "canada_per_capita_income.csv"
x_raw, y = load_dataset(csv_path)
print(f"Loaded {x_raw.shape[0]} rows from {csv_path}")

Loaded 47 rows from canada_per_capita_income.csv


## 2. Model equations (univariate linear regression)
- Hypothesis: $\hat{y} = w_0 + w_1 x$
- Cost (MSE): $J(w_0, w_1) = \dfrac{1}{2m} \sum_{i=1}^{m} (\hat{y}^{(i)} - y^{(i)})^2$
- Gradients:
  - $\dfrac{\partial J}{\partial w_0} = \dfrac{1}{m} \sum_{i=1}^{m} (\hat{y}^{(i)} - y^{(i)})$
  - $\dfrac{\partial J}{\partial w_1} = \dfrac{1}{m} \sum_{i=1}^{m} (\hat{y}^{(i)} - y^{(i)}) x^{(i)}$
- Updates (gradient descent):
  - $w_0 \leftarrow w_0 - \alpha \dfrac{\partial J}{\partial w_0}$
  - $w_1 \leftarrow w_1 - \alpha \dfrac{\partial J}{\partial w_1}$

In [3]:
def normalize_feature(x):
    """
    Normalize feature to zero mean and unit variance for stable gradient descent.
    Returns normalized x and the (mean, std) for inverse transforms.
    """
    mean = np.mean(x)
    std = np.std(x) + 1e-8
    x_norm = (x - mean) / std
    return x_norm, mean, std


def add_bias(x):
    """
    Add bias column of ones to feature matrix.
    """
    return np.hstack([np.ones((x.shape[0], 1)), x])


def predict(X, w):
    """Compute predictions given design matrix X and weights w."""
    return X @ w


def compute_cost(X, y, w):
    """Mean squared error cost: J = (1/(2m)) * sum((Xw - y)^2)."""
    m = X.shape[0]
    residual = predict(X, w) - y
    return (1.0 / (2 * m)) * np.sum(residual ** 2)


def compute_gradient(X, y, w):
    """Gradient of MSE: (1/m) * X^T (Xw - y)."""
    m = X.shape[0]
    residual = predict(X, w) - y
    return (1.0 / m) * (X.T @ residual)


def gradient_descent(X, y, alpha=0.01, num_iters=1000, verbose=False):
    """
    Batch gradient descent for linear regression.
    Returns learned weights and cost history.
    """
    w = np.zeros((X.shape[1], 1))
    costs = []
    for i in range(num_iters):
        grad = compute_gradient(X, y, w)
        w -= alpha * grad
        if verbose and i % 100 == 0:
            cost = compute_cost(X, y, w)
            costs.append(cost)
            print(f"Iter {i:4d} | Cost {cost:.6f}")
        elif not verbose:
            costs.append(compute_cost(X, y, w))
    return w, costs

## 3. Train the model

In [ ]:
# Normalize feature, build design matrix, and run gradient descent
x_norm, x_mean, x_std = normalize_feature(x_raw)
X = add_bias(x_norm)

alpha = 0.1
num_iters = 1500

w, costs = gradient_descent(X, y, alpha=alpha, num_iters=num_iters, verbose=True)

print("Training complete.")
print(f"Final cost: {costs[-1]:.6f}")

## 4. Evaluate and visualize

In [ ]:
def denormalize_weights(w, mean, std):
    """
    Convert weights learned on normalized x back to original scale.
    Returns slope and intercept in original units.
    """
    w0, w1 = w.flatten()
    slope = w1 / std
    intercept = w0 - (w1 * mean / std)
    return intercept, slope


# Recover slope/intercept in original scale
intercept, slope = denormalize_weights(w, x_mean, x_std)
print(f"Model: income = {slope:.4f} * year + {intercept:.4f}")

# Predictions for training points
pred_train = predict(X, w)

# Metrics
m = y.shape[0]
rmse = np.sqrt((1.0 / m) * np.sum((pred_train - y) ** 2))
mae = (1.0 / m) * np.sum(np.abs(pred_train - y))
print(f"RMSE: {rmse:.4f}")
print(f"MAE:  {mae:.4f}")

# Plot data and regression line
plt.figure(figsize=(8, 5))
plt.scatter(x_raw, y, color="steelblue", label="Data")

# Line over feature range
x_line = np.linspace(x_raw.min(), x_raw.max(), 100).reshape(-1, 1)
x_line_norm = (x_line - x_mean) / x_std
X_line = add_bias(x_line_norm)
y_line = predict(X_line, w)

plt.plot(x_line, y_line, color="crimson", linewidth=2.5, label="Fitted line")
plt.xlabel("Year")
plt.ylabel("Per Capita Income")
plt.title("Linear Regression Fit")
plt.legend()
plt.tight_layout()
plt.show()

# Plot cost over iterations
plt.figure(figsize=(8, 4))
plt.plot(costs, color="darkgreen", linewidth=2)
plt.xlabel("Iteration")
plt.ylabel("Cost (MSE * 0.5)")
plt.title("Cost vs. iterations")
plt.tight_layout()
plt.show()